In [14]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

import pandas as pd
import pickle

## Import the data
We start by import the data created in the data_collection notebook. This can then be split for training, validation and testing

In [15]:
# Import csv
df = pd.read_csv('data.csv')

# Split data into X and y data
X = df.drop(columns = ['0']).copy()
y = df['0']

# Split data into testing, training and validation data
X_train, X_rem, y_train, y_rem = train_test_split(X, y, train_size=0.8)
X_val, X_test, y_val, y_test = train_test_split(X_rem,y_rem, test_size=0.5)

## Fit model 
We use a random forest for multiclass classification of the signs.

In [16]:
rf = RandomForestClassifier(class_weight = 'balanced_subsample')
rf.fit(X_train, y_train)
print("Random forest's test score:", rf.score(X_test, y_test))

Random forest's test score: 1.0


## 

In [17]:
pickle.dump(rf, open('random_forest.sav', 'wb'))

## Fit the model to our data
10 epochs were run since it was tested to have a good balance of trainng and avoiding overfitting